In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys

from autocorrect import spell
sys.path.append("../..") # go to parent dir

In [3]:
from aspects.analysis.statistics_bing_liu import (load_all_datasets)
from aspects.preprocessing.preprocesser import Preprocesser

In [4]:
import matplotlib.pyplot as plt
import pandas as pd

from typing import List, Iterable
from tqdm import tqdm
from datetime import datetime

from nltk.corpus import wordnet
from wordfreq import zipf_frequency

import spacy

from tqdm import tnrange, tqdm_notebook
import jellyfish
%matplotlib inline

In [5]:
nlp = spacy.load('en')

In [6]:
processer = Preprocesser()

# Get datasets and load 

In [7]:
datasets = load_all_datasets()

In [8]:
# similar_aspects = {
#     dataset_name: get_most_similar_text_pairs(dataset.aspect.tolist())
#     for dataset_name, dataset in datasets.items()
# }
# df = pd.DataFrame.from_dict(similar_aspects)
# df.to_csv('similar_aspects_bing_liu_datasets.csv')
# for dataset_name, similarity_scores in similar_aspects.items():
#     print(sorted([(score, pair) for pair, score in similarity_scores.items()], reverse=True))

In [9]:
df = pd.read_csv('similar_aspects_bing_liu_datasets.csv')

In [10]:
df.sample(10)

,,Canon PowerShot SD500.txt,Canon S100.txt,Diaper Champ.txt,Hitachi router.txt,Linksys Router.txt,MicroMP3.txt,Nokia 6600.txt,ipod.txt,norton.txt
363,radio feature ## radio ...,,,,,,0.8769230769230769,,,NaN
445,start ## stable ...,,,,0.79,,,,,NaN
317,picture of people ## picture detail ...,,0.8605042016806722,,,,,,,NaN
458,symantec ## symantect product ...,,,,,,,,,0.888889
200,instruction ## construct ...,,,,,,0.7575757575757575,,,NaN
97,connection ## disconnect ...,,,,,0.8641025641025641,,,,NaN
383,ring ## ringer ...,,,,,,,0.9142857142857143,,NaN
321,plaeyr ## playlist ...,,,,,,0.7925925925925926,,,NaN
224,macro mode ## auto mode ...,,0.825925925925926,,,,,,,NaN
425,software ## pc software ...,,,,,,0.8888888888888888,,,NaN


# Wordnet and zipf

In [11]:
zipf_aspects_df = {
    dataset_name: pd.DataFrame([
        (word, zipf_frequency(word, 'en'), spell(word)) for word in set(dataset.aspect.tolist())], 
        columns=['word', 'zipf', 'spelled'])
    for dataset_name, dataset in datasets.items()
} 

In [15]:
# for col in zipf_aspects_df:
#     print(zipf_aspects_df[col][zipf_aspects_df[col].word != zipf_aspects_df[col].spelled].sort_values('zipf'))

In [17]:
for col in zipf_aspects_df:
    print(zipf_aspects_df[col][zipf_aspects_df[col].zipf > 5].sort_values('zipf'))

        word  zipf  spelled
54     build  5.01    build
89    weight  5.03   weight
14     radio  5.03    radio
23    access  5.05   access
127    built  5.06    built
16     style  5.08    style
69   quality  5.10  quality
140    stand  5.12    stand
33      size  5.14     size
109   player  5.14   player
203   design  5.15   design
78     sound  5.16    sound
5      price  5.22    price
13      cost  5.22     cost
112    space  5.24    space
167    looks  5.31    looks
160  working  5.45  working
41   support  5.48  support
166    small  5.53    small
173     read  5.55     read
122    power  5.55    power
105     case  5.60     case
37      feel  5.66     feel
20      look  5.79     look
200      off  5.92      off
175     made  5.92     made
79      back  6.05     back
70      make  6.06     make
101       on  6.91       on
            word  zipf      spelled
25           hot  5.13          hot
18          size  5.14         size
19   performance  5.14  performance
103       design

In [13]:
for synsets in wordnet.synsets('reading', pos='n'):
    print(synsets)

Synset('reading.n.01')
Synset('reading.n.02')
Synset('reading.n.03')
Synset('reading.n.04')
Synset('interpretation.n.01')
Synset('reading.n.06')
Synset('recitation.n.02')
Synset('reading.n.08')
